Design and implement in code a Planner algorithm for a square chessboard of a given size (MxM) with \
certain cells marked red as restricted areas. 100x100 will be the maximum chessboard size. There are N \
kings on the board, each with a specific target cell to reach. The kings move sequentially, one by one, one \
step at a time (K1 step1, K2 step1…. K1 step2, K2 step2…), and must adhere to two main rules: they \
cannot move into restricted cells, and they cannot end a move adjacent to another king (including \
diagonally). A king may stay in place (empty move). The task is to compute a path for each king to its \
target, ensuring compliance with the movement rules and the sequential order of moves. \
Planner’s input would be given as 2 files: \
File1 - king starting and target positions \
x_from, y_from, x_to, y_to \
… \
File2 - chessboard size and restricted cells \
M \
x, y \
…\
x and y - integer coordinates of restricted cells - start from zero. \
If plan (solution) does not exist, Planner should detect and output that plan does not exist.\
If plan does exist, Planner should output a plan in a format \
x_from, y_from, x_to, y_to \
… \
where each line is one step of one king, and king_id would be deduced by a test program from “from” \
coordinates. 

In [165]:
import numpy as np
import math
import heapq
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [166]:
def get_map(test_case):
    
    map_file = open('test_cases/' + str(test_case) + '/map.txt', 'r')
    map_file = str(map_file.read())
    lines = map_file.strip().split('\n')
    obstacles = [tuple(line.split(',')) for line in lines[1:]]
    obstacles = [(int(obstacle[0]),int(obstacle[1])) for obstacle in obstacles]
    grid_size = (int(map_file[0]), int(map_file[0]))
    
    return grid_size, obstacles


def get_kings(test_case):
    '''
    This function returns the start and 
    the goal position of the kings
    as a list of tuples' tuples
    '''
    kings_file = open('test_cases/' + str(test_case) + '/kings.txt', 'r')
    kings_file = str(kings_file.read())
    start_goal = kings_file.strip().split('\n')
    total_no_kings = len(start_goal)

    kings = np.zeros(total_no_kings, dtype=object)

    for i, king in enumerate(start_goal):
        king_i = king.split(',')
        start = (int(king_i[0]),int(king_i[1]))
        goal = (int(king_i[2]),int(king_i[3]))
        kings[i] = (start, goal)

    return kings

# A* Algorithm

In [167]:
class AStarPathfinder:
    def __init__(self, grid_size, start_pos, goal_pos, obstacles):
        self.grid_size = grid_size
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.obstacles = obstacles
        self.grid = np.zeros(grid_size, dtype=np.int8)
        for obs in obstacles:
            self.grid[obs] = 1  # Mark obstacles on the grid

    def heuristic(self, a, b):
        """Calculate the Manhattan distance from point a to point b."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def neighbors(self, node):
        """Generate the neighbors of a given node."""
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # 4 possible movements
        result = []
        for dx, dy in directions:
            nx, ny = node[0] + dx, node[1] + dy
            if 0 <= nx < self.grid_size[0] and 0 <= ny < self.grid_size[1]:
                if self.grid[nx, ny] == 0:  # Check if it's not an obstacle
                    result.append((nx, ny))
        return result

    def a_star_search(self):
        """Perform the A* search algorithm."""
        open_set = []
        heapq.heappush(open_set, (0 + self.heuristic(self.start_pos, self.goal_pos), self.start_pos))
        came_from = {}
        g_score = {self.start_pos: 0}
        f_score = {self.start_pos: self.heuristic(self.start_pos, self.goal_pos)}

        while open_set:
            current = heapq.heappop(open_set)[1]

            if current == self.goal_pos:
                return self.reconstruct_path(came_from, current)

            for neighbor in self.neighbors(current):
                tentative_g_score = g_score[current] + 1  # step cost is always 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, self.goal_pos)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

        return None

    def reconstruct_path(self, came_from, current):
        """Reconstruct the path from start to goal."""
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        return total_path[::-1]  # Return reversed path

# Usage example
# grid_size = (4, 4)
# start_pos = (1, 3)
# goal_pos = (3, 1)
# obstacles = [(0,2), (2,2), (2,1)]
# pathfinder = AStarPathfinder(grid_size, start_pos, goal_pos, obstacles)
# path = pathfinder.a_star_search()
# print(path)



In [168]:
test_case = 1
grid_size, obstacles = get_map(test_case)
kings = get_kings(test_case)
total_no_kings = len(kings)

def get_initial_paths(kings, obstacles):

    all_paths = np.zeros(total_no_kings, dtype=object)
    all_path_costs = np.zeros(total_no_kings, dtype=int)

    for i, king in enumerate(kings):
        pathfinder = AStarPathfinder(grid_size, king[0], king[1], obstacles)
        path = pathfinder.a_star_search()
        # print('King_'+str(i), path)
        all_path_costs[i] = len(path)
        all_paths[i] = np.array(path)

    return all_paths, all_path_costs

# Conflict-Based Search (High Level)

In [170]:
init_paths, all_path_costs = get_initial_paths(kings, obstacles)
SIC = np.sum(all_path_costs)

print(all_path_costs)
print(SIC)

[ 7 10 13  7 11  9 10  8]
75
